In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import librosa
import wave
import pandas as pd
import soundfile as sf
import librosa.display
from PIL import Image
import os
import tensorflow as tf
import scipy.signal as signal

cutoff_frequency = 195
sampling_rate=44100

In [2]:
def apply_low_pass_filter(audio, sampling_rate, cutoff_freq):
    nyquist_freq = 0.5 * sampling_rate
    normalized_cutoff_freq = cutoff_freq / nyquist_freq
    b, a = signal.butter(4, normalized_cutoff_freq, btype='low', analog=False)
    denoised_audio = signal.lfilter(b, a, audio)
    return denoised_audio

In [ ]:
def downsample_audio(audio,original_sampling_rate,target_sampling_rate):
    resampled_audio = librosa.resample(audio, orig_sr=original_sampling_rate, target_sr=target_sampling_rate)
    return resampled_audio

In [ ]:
def split_audio(audio, segment_length):
    num_segments = len(audio) // segment_length
    segments = [audio[i*segment_length:(i+1)*segment_length] for i in range(num_segments)]
    return segments

In [ ]:
time_shift_factor = 1
pitch_shift_steps = -0.5

def apply_pitch_shift(audio, target_sampling_rate, pitch_shift):
    audio_float = audio.astype(np.float32)  # Remove the normalization step
    shifted_audio = librosa.effects.pitch_shift(audio_float, sr=target_sampling_rate, n_steps=pitch_shift)
    return shifted_audio

def apply_time_shift(audio, time_shift):
    shifted_audio = np.roll(audio, int(time_shift))  # Convert the time shift to an integer
    return shifted_audio

In [ ]:
#Augmented-A

def audioAugment(root_directory, output_folder):
    # Iterate through subdirectories and files in the root directory
    for folder_name, subfolders, filenames in os.walk(root_directory):
        # Iterate through audio files in the current folder

        for filename in filenames:
            # Check if the file has an audio extension (e.g., .wav, .mp3, etc.)
            if filename.lower().endswith(('.wav')):
                # Construct the full path to the audio file
                audio_path = os.path.join(folder_name, filename)
                
                audio, sampling_rate = librosa.load(audio_path, sr=None)

                # Denoising
                cutoff_frequency = 195
                denoised_audio = apply_low_pass_filter(audio, sampling_rate, cutoff_frequency)

                # Downsampling
                target_sampling_rate = sampling_rate // 10
                downsampled_audio = downsample_audio(denoised_audio, sampling_rate, target_sampling_rate)

                # Splitting audio
                segment_length = target_sampling_rate * 3
                segments = split_audio(downsampled_audio, segment_length)
                
                time_shifted_segments = [apply_time_shift(segment, time_shift_factor) for segment in segments]

                for i, segment in enumerate(time_shifted_segments):
                    output_path=os.path.join(output_folder,'time_{}_{}'.format(i,filename))
                    sf.write(output_path, segment, target_sampling_rate)
                    
                pitch_shifted_segments = [apply_pitch_shift(segment, target_sampling_rate, pitch_shift_steps) for segment in segments]
                
                for i, segment in enumerate(pitch_shifted_segments):
                    output_path=os.path.join(output_folder,'pitch_{}_{}'.format(i,filename))
                    sf.write(output_path, segment, target_sampling_rate)

In [ ]:
import os
from scipy.io import wavfile
import soundfile as sf

def audioSplit(root_directory, output_folder):
    # Iterate through audio files in the root directory
    for folder_name, subfolders, filenames in os.walk(root_directory):
        # Check if the file has an audio extension (e.g., .wav, .mp3, etc.)
        for filename in filenames:
            
            if filename.lower().endswith(('.wav')):
                # Construct the full path to the audio file
                audio_path = os.path.join(root_directory, filename)

                audio, sampling_rate = librosa.load(audio_path, sr=None)

                # Denoising
                cutoff_frequency = 195
                denoised_audio = apply_low_pass_filter(audio, sampling_rate, cutoff_frequency)

                # Downsampling
                target_sampling_rate = sampling_rate // 10
                downsampled_audio = downsample_audio(denoised_audio, sampling_rate, target_sampling_rate)

                # Splitting audio
                segment_length = target_sampling_rate * 3
                segments = split_audio(downsampled_audio, segment_length)
        
                # Write the segments to the output folder
                for i, segment in enumerate(segments):
                    segment_filename = f"segment_{i}_{filename}"
                    segment_path = os.path.join(output_folder, segment_filename)
                    sf.write(segment_path, segment, target_sampling_rate)

In [ ]:
#Augmented-A & Split-A/Atraining_artifact
root_directory='/Users/rishikabethi/Desktop/Heart/Dataset_A/Atraining_artifact'
output_folder='/Users/rishikabethi/Desktop/Heart/Dataset_A/Augmented-A/Atraining_artifact'
output_folder1='/Users/rishikabethi/Desktop/Heart/Dataset_A/Split-A/Atraining_artifact'
audioAugment(root_directory, output_folder)
audioSplit(root_directory, output_folder1)

In [ ]:
#Augmented-A & Split-A/Atraining_extrahls
root_directory='/Users/rishikabethi/Desktop/Heart/Dataset_A/Atraining_extrahls'
output_folder='/Users/rishikabethi/Desktop/Heart/Dataset_A/Augmented-A/Atraining_extrahls'
output_folder1='/Users/rishikabethi/Desktop/Heart/Dataset_A/Split-A/Atraining_extrahls'
audioAugment(root_directory, output_folder)
audioSplit(root_directory, output_folder1)

In [ ]:
#Augmented-A & Split-A/Atraining_murmur
root_directory='/Users/rishikabethi/Desktop/Heart/Dataset_A/Atraining_murmur'
output_folder='/Users/rishikabethi/Desktop/Heart/Dataset_A/Augmented-A/Atraining_murmur'
output_folder1='/Users/rishikabethi/Desktop/Heart/Dataset_A/Split-A/Atraining_murmur'
audioAugment(root_directory, output_folder)
audioSplit(root_directory, output_folder1)

In [ ]:
#Augmented-A & Split-A/Atraining_normal
root_directory='/Users/rishikabethi/Desktop/Heart/Dataset_A/Atraining_normal'
output_folder='/Users/rishikabethi/Desktop/Heart/Dataset_A/Augmented-A/Atraining_normal'
output_folder1='/Users/rishikabethi/Desktop/Heart/Dataset_A/Split-A/Atraining_normal'
audioAugment(root_directory, output_folder)
audioSplit(root_directory, output_folder1)

In [ ]:
def time_warp(spec, W=5):
    num_bins, num_frames = spec.shape
    spec_aug = np.copy(spec)

    # Generate random control points for time warping
    control_points = [(0, 0), (num_bins-1, num_frames-1)]
    for _ in range(W):
        random_point = np.random.randint(1, num_bins-1)
        control_points.append((random_point, np.random.randint(0, num_frames-1)))

    # Apply time warping
    for i in range(len(control_points)-1):
        c1, c2 = control_points[i], control_points[i+1]
        spec_aug[c1[0]:c2[0]+1, :] = np.roll(spec[c1[0]:c2[0]+1, :], c2[1] - c1[1], axis=1)

    return spec_aug

def frequency_mask(spec, F=30, num_masks=2):
    spec_aug = np.copy(spec)

    # Apply frequency masking
    num_bins, num_frames = spec.shape
    for _ in range(num_masks):
        f = np.random.randint(0, F)
        f0 = np.random.randint(0, num_bins - f)
        spec_aug[f0:f0+f, :] = 0

    return spec_aug

In [ ]:
#SpecAugment-A
def SpecAugment(root_directory, output_folder):
    # Iterate through subdirectories and files in the root directory
    for folder_name, subfolders, filenames in os.walk(root_directory):
        # Iterate through audio files in the current folder

        for filename in filenames:
            if filename.lower().endswith(('.wav')):
#                 Construct the full path to the audio file
                audio_path = os.path.join(folder_name, filename)
                # Load audio file
                audio, sr = librosa.load(audio_path, sr=None)
                
                target_sampling_rate=sr//10
                # Compute spectrogram
                spectrogram = librosa.feature.melspectrogram(y=audio, sr=target_sampling_rate, n_fft=2048, hop_length=512, n_mels=128)
                spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
                
                # Normalize the spectrogram to values between 0 and 1
                spectrogram_normalized = (spectrogram_db - np.min(spectrogram_db)) / (np.max(spectrogram_db) - np.min(spectrogram_db))

                # Scale the spectrogram to values between 1 and 255
                spectrogram_scaled = spectrogram_normalized * 255
                
                # Apply time warping
                time_warped_spectrogram = time_warp(spectrogram_scaled)

                # Generate frequency-masked spectrograms
                frequency_masked_spectrogram_1 = frequency_mask(spectrogram_scaled)
                frequency_masked_spectrogram_2 = frequency_mask(spectrogram_scaled)
                
                fig, ax = plt.subplots(figsize=(1.28,1.28))
                
                img1=librosa.display.specshow(time_warped_spectrogram, sr=sr)
                plt.savefig(os.path.join(output_folder,'time_{}.png'.format(filename[:-4])),transparent=True)
                
                img2=librosa.display.specshow(frequency_masked_spectrogram_1, sr=sr)
                plt.savefig(os.path.join(output_folder,'freq_{}.png'.format(filename[:-4])),transparent=True)
                
                img3=librosa.display.specshow(frequency_masked_spectrogram_2, sr=sr)
                plt.savefig(os.path.join(output_folder,'freq2_{}.png'.format(filename[:-4])),transparent=True)
                
                img4=librosa.display.specshow(spectrogram_scaled, sr=sr)
                plt.savefig(os.path.join(output_folder,'spec_{}.png'.format(filename[:-4])),transparent=True)
                
                plt.close()

In [ ]:
#SpecAugment/Atraining_artifact

root_directory='/Users/rishikabethi/Desktop/Heart/Dataset_A/Split-A/Atraining_artifact'
output_folder='/Users/rishikabethi/Desktop/Heart/Dataset_A/SpecAugment-A/Atraining_artifact'
SpecAugment(root_directory, output_folder)

In [ ]:
#SpecAugment/Atraining_extrahls

root_directory='/Users/rishikabethi/Desktop/Heart/Dataset_A/Split-A/Atraining_extrahls'
output_folder='/Users/rishikabethi/Desktop/Heart/Dataset_A/SpecAugment-A/Atraining_extrahls'
SpecAugment(root_directory, output_folder)

In [ ]:
#SpecAugment/Atraining_murmur

root_directory='/Users/rishikabethi/Desktop/Heart/Dataset_A/Split-A/Atraining_murmur'
output_folder='/Users/rishikabethi/Desktop/Heart/Dataset_A/SpecAugment-A/Atraining_murmur'
SpecAugment(root_directory, output_folder)

In [ ]:
#SpecAugment/Atraining_normal

root_directory='/Users/rishikabethi/Desktop/Heart/Dataset_A/Split-A/Atraining_normal'
output_folder='/Users/rishikabethi/Desktop/Heart/Dataset_A/SpecAugment-A/Atraining_normal'
SpecAugment(root_directory, output_folder)

In [ ]:
#Spec

def Spec(root_directory, output_folder):
    # Iterate through subdirectories and files in the root directory
    for folder_name, subfolders, filenames in os.walk(root_directory):
        # Iterate through audio files in the current folder

        for filename in filenames:
            if filename.lower().endswith(('.wav')):
#                 Construct the full path to the audio file
                audio_path = os.path.join(folder_name, filename)
                # Load audio file
                audio, sr = librosa.load(audio_path, sr=None)
                
                target_sampling_rate=sr//10
                # Compute spectrogram
                spectrogram = librosa.feature.melspectrogram(y=audio, sr=target_sampling_rate, n_fft=2048, hop_length=512, n_mels=128)
                spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
                
                fig, ax = plt.subplots(figsize=(1.28,1.28))
                
                img1=librosa.display.specshow(spectrogram_db, sr=sr)
                plt.savefig(os.path.join(output_folder,'spec_{}.png'.format(filename[:-4])),transparent=True)
                
                plt.close()

In [ ]:
#Spec/Atraining_artifact

root_directory='/Users/rishikabethi/Desktop/Heart/Dataset_A/Augmented-A/Atraining_artifact'
output_folder='/Users/rishikabethi/Desktop/Heart/Dataset_A/Spec-A/Atraining_artifact'
Spec(root_directory, output_folder)

In [ ]:
#Spec/Atraining_extrahls

root_directory='/Users/rishikabethi/Desktop/Heart/Dataset_A/Augmented-A/Atraining_extrahls'
output_folder='/Users/rishikabethi/Desktop/Heart/Dataset_A/Spec-A/Atraining_extrahls'
Spec(root_directory, output_folder)

In [ ]:
#Spec/Atraining_murmur

root_directory='/Users/rishikabethi/Desktop/Heart/Dataset_A/Augmented-A/Atraining_murmur'
output_folder='/Users/rishikabethi/Desktop/Heart/Dataset_A/Spec-A/Atraining_murmur'
Spec(root_directory, output_folder)

In [ ]:
#Spec/Atraining_normal

root_directory='/Users/rishikabethi/Desktop/Heart/Dataset_A/Augmented-A/Atraining_normal'
output_folder='/Users/rishikabethi/Desktop/Heart/Dataset_A/Spec-A/Atraining_normal'
Spec(root_directory, output_folder)